In [ ]:
import geopandas as gpd
import rasterio
from pathlib import Path
from collections import deque
from rasterio.transform import rowcol
from rasterio.mask import mask
from google.colab import drive
from osgeo import gdal
import numpy as np

# Mount Google drive to colab runtime.
drive.mount('/content/drive')
drive_path = Path(f'/content/drive/My Drive/')

# Input Parameters

In [ ]:
FARM_LOCATION = ''
MWS_OF_FARM = ''
DEPRESSIONLESS_DEM = ''
HYDRAULIC_CONDUCTIVITY = ''
OUTPUT_FOLDER_PATH = ''

# Utility Functions

In [ ]:
def load_shapefiles(point_path, polygon_path):
    """
    Load point (Farm Location) and polygon (Microwatershed boundary) shapefiles
    and align their Coordinate Reference Systems (CRS).
    
    Parameters:
        point_path (str): File path to the point shapefile.
        polygon_path (str): File path to the polygon shapefile.
    
    Returns:
        tuple: GeoDataFrames of points and polygons with aligned CRS.
    """
    point_gdf = gpd.read_file(point_path)
    polygon_gdf = gpd.read_file(polygon_path)
    
    # Align CRS
    if point_gdf.crs != polygon_gdf.crs:
        point_gdf = point_gdf.to_crs(polygon_gdf.crs)
    
    return point_gdf, polygon_gdf

def get_containing_polygon(point_gdf, polygon_gdf):
    """
    Identify and return the polygon geometry that contains a given point.
    
    Assumes only one point in point_gdf.

    Parameters:
        point_gdf (GeoDataFrame): GeoDataFrame containing a single point.
        polygon_gdf (GeoDataFrame): GeoDataFrame containing polygon geometries.
    
    Returns:
        dict or None: The geometry of the containing polygon in GeoJSON format, or None if no match.
    """
    # Extract the first (and assumed only) point geometry
    point = point_gdf.geometry.iloc[0]
    
    # Filter polygons that contain the point
    containing = polygon_gdf[polygon_gdf.contains(point)]
    
    # Return None if no containing polygon is found
    if containing.empty:
        return None

    # Return the containing polygon geometry as GeoJSON
    return containing.geometry.iloc[0].__geo_interface__


def clip_raster_with_polygon(raster_path, polygon_geom):
    """
    Clip a raster file using the given polygon geometry.

    Parameters:
        raster_path (str): Path to the input raster file.
        polygon_geom (dict): Polygon geometry in GeoJSON-like format.

    Returns:
        tuple: A tuple containing:
            - out_image (np.ndarray): Clipped raster data.
            - out_meta (dict): Updated raster metadata.
            - nodata_value (float): NoData value used in the raster.
    """
    with rasterio.open(raster_path) as src:
        # Clip raster using the polygon geometry
        out_image, out_transform = mask(src, [polygon_geom], crop=True)
        
        # Copy and update metadata for the output raster
        out_meta = src.meta.copy()
        nodata_value = src.nodata if src.nodata is not None else 0

    # Update metadata with new dimensions and transform
    out_meta.update({
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })
    
    return out_image, out_meta, nodata_value


def save_raster(output_path, out_image, out_meta):
    """
    Save a raster array to disk with given metadata.

    Parameters:
        output_path (str): Destination path for the output raster.
        out_image (np.ndarray): Raster image array to save.
        out_meta (dict): Metadata for the output raster.
    """
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)


def get_slope(dem=None, dem_path=None):
    """
    Compute slope in degrees from a Digital Elevation Model (DEM).

    Parameters:
        dem (gdal.Dataset, optional): GDAL raster dataset object.
        dem_path (str, optional): File path to the DEM if 'dem' is not provided.

    Returns:
        np.ndarray: 2D array representing slope in degrees.
    """
    # Load DEM from file path if not directly provided
    if dem is None:
        if dem_path is None:
            raise ValueError("Either 'dem' or 'dem_path' must be provided.")
        else:
            dem = gdal.Open(dem_path)
    
    if dem is None:
        raise FileNotFoundError(f"DEM file not found: {dem_path}")

    # Perform slope calculation in memory using GDAL
    slope_ds = gdal.DEMProcessing('', dem, 'slope', format='MEM', slopeFormat='degree')

    # Read the slope band into a NumPy array
    slope_array = slope_ds.GetRasterBand(1).ReadAsArray()

    return slope_array


def get_row_col_from_point_gdf(point_gdf, transform):
    """
    Get (row, col) index in raster array for the first point in point_gdf.

    Parameters:
        - point_gdf: GeoDataFrame with a single point geometry
        - transform: Affine transform of the raster

    Returns:
        - (row, col): Tuple of raster array indices
    """
    if point_gdf.empty:
        raise ValueError("The point GeoDataFrame is empty.")
    point_geom = point_gdf.geometry.iloc[0]
    row, col = rowcol(transform, point_geom.x, point_geom.y)
    return row, col


def compute_flow_transfer_cost(slope_array, k_array, slope_nodata=None, k_nodata=None):
    """
    Compute pixel-wise: 30 / (K * slope)
    Inputs:
        - slope_array: NumPy array of slope values
        - k_array: NumPy array of K values
        - slope_nodata: optional nodata value for slope
        - k_nodata: optional nodata value for K
    Returns:
        - result_array: NumPy array with computed values
        - mask: Boolean array where output is valid
    """
    assert slope_array.shape == k_array.shape, "Input arrays must have the same shape"

    # Create mask for invalid pixels
    mask = np.ones_like(slope_array, dtype=bool)
    if slope_nodata is not None:
        mask &= (slope_array != slope_nodata)
    if k_nodata is not None:
        mask &= (k_array != k_nodata)
    mask &= (slope_array != 0) & (k_array != 0)

    # Initialize output array
    result = np.full_like(slope_array, fill_value=np.nan, dtype=np.float32)

    # Compute ratio where valid
    with np.errstate(divide='ignore', invalid='ignore'):
        result[mask] = 30.0 / (k_array[mask] * slope_array[mask])

    return result, mask


def get_upstream_catchment_path_distance(elevation, ft_cost, start_row, start_col, threshold=120):
    """
    Identify upstream catchment area based on elevation and a cost threshold using path-based traversal.
    
    Parameters:
        elevation (np.ndarray): 2D array of elevation values.
        ft_cost (np.ndarray): 2D array representing foot travel cost at each cell.
        start_row (int): Row index of the starting point (typically the pour point).
        start_col (int): Column index of the starting point.
        threshold (float): Maximum cumulative foot travel cost to allow for upstream expansion.

    Returns:
        np.ndarray: Binary mask of the upstream catchment area (1 = included, 0 = excluded).
    """
    rows, cols = elevation.shape
    
    # Track visited cells to avoid repeated processing
    visited = np.zeros_like(elevation, dtype=bool)
    
    # Output mask for catchment area
    catchment = np.zeros_like(elevation, dtype=np.uint8)

    # Define relative indices for 8-neighborhood (diagonal + orthogonal)
    neighbors = [(-1, -1), (-1, 0), (-1, 1),
                 (0, -1),          (0, 1),
                 (1, -1),  (1, 0),  (1, 1)]

    # Initialize BFS queue with the starting point and zero cost
    queue = deque()
    queue.append((start_row, start_col, 0))
    
    visited[start_row, start_col] = True
    catchment[start_row, start_col] = 1

    # Breadth-First Search traversal with cost-based constraint
    while queue:
        r, c, curr_ft_cost = queue.popleft()

        # Stop traversal if cumulative cost exceeds the threshold
        if curr_ft_cost >= threshold:
            continue

        for dr, dc in neighbors:
            nr, nc = r + dr, c + dc
            # Check bounds and whether cell is already visited
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr, nc]:
                # Only move to upstream (equal or higher elevation) cells
                if elevation[nr, nc] >= elevation[r, c]:
                    visited[nr, nc] = True
                    catchment[nr, nc] = 1
                    queue.append((nr, nc, curr_ft_cost + ft_cost[nr, nc]))

    return catchment


# Upstream Catchment Delineation
This block performs the full upstream catchment delineation for a given farm location. It first identifies the containing microwatershed and clips relevant raster layers (DEM and hydraulic conductivity) to this boundary. It then computes slope and flow transfer cost for each pixel, locates the farm in raster space, and identifies all upstream cells using a cost-constrained path traversal. Finally, it saves the resulting upstream catchment as a raster file.

In [ ]:
# Load farm location and microwatershed (MWS) shapefiles
farm_location_gdf, mws_gdf = load_shapefiles(FARM_LOCATION, MWS_OF_FARM)

# Identify the microwatershed polygon containing the farm location
containing_mws_geom = get_containing_polygon(farm_location_gdf, mws_gdf)

# Exit early if farm location is not within any microwatershed
if containing_mws_geom is None:
    print("No mws contains the farm location.")
    exit()

# Clip the depressionless DEM to the containing microwatershed geometry
dem, dem_meta, dem_nodata = clip_raster_with_polygon(DEPRESSIONLESS_DEM, containing_mws_geom)

# Compute slope from the clipped DEM
slope = get_slope(dem)

# Clip the hydraulic conductivity raster to the same microwatershed
k_array, k_meta, k_nodata = clip_raster_with_polygon(HYDRAULIC_CONDUCTIVITY, containing_mws_geom)

# Compute flow transfer cost based on slope and hydraulic conductivity
ft_cost, mask = compute_flow_transfer_cost(slope, k_array, dem_nodata, k_nodata)

# Get the row and column index of the farm point in the raster grid
transform = dem_meta['transform']
row, col = get_row_col_from_point_gdf(farm_location_gdf, transform)

# Identify upstream catchment area using elevation and cost constraints
catchment = get_upstream_catchment_path_distance(dem, ft_cost, row, col)

# Define output file path for saving the catchment raster
output_path = Path(OUTPUT_FOLDER_PATH) / 'upstream_catchment.tif'

# Prepare metadata for saving the catchment raster
catchment_meta = dem_meta.copy()
catchment_meta.update({
    'dtype': 'uint8',   # Since catchment is a binary mask
    'nodata': 0         # Use 0 to represent absence of catchment
})

# Save the catchment area as a raster file
save_raster(output_path, catchment, catchment_meta)
